# Setup

In [ ]:
videos_dir = None
save_dir = None

In [ ]:
import os

dir_path = os.getcwd()

if not videos_dir:
    if 'google.colab' in str(get_ipython()):
        # Update this path as necessary
        videos_dir = f'{dir_path}/AIC_Video'
    elif 'kaggle' in str(get_ipython()):
        videos_dir = f'{dir_path}/AIC_Video'
    else:
        parent_dir_path = os.path.dirname(dir_path)
        videos_dir = f'{parent_dir_path}/dataset/AIC_Video'
        
if not save_dir:
    save_dir = f'{dir_path}/SceneJSON'

In [ ]:
!git clone https://github.com/soCzech/TransNetV2.git
!git lfs install
!cd TransNetV2
!git lfs fetch https://github.com/soCzech/TransNetV2.git
!git lfs checkout

In [ ]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg
import torch
import numpy as np
from tqdm import tqdm
from TransNetV2.inference.transnetv2 import TransNetV2

# Parse video info

In [ ]:
def parse_video_info(videos_dir):
    """Parse video information from the directory structure."""
    all_video_paths = {}
    for part in sorted(os.listdir(videos_dir)):
        data_part = part.split('_')[-1]  # L01, L02 for example
        all_video_paths[data_part] = {}

    for data_part in sorted(all_video_paths.keys()):
        data_part_path = f'{videos_dir}/Videos_{data_part}/video'
        video_paths = sorted(os.listdir(data_part_path))
        video_ids = [video_path.replace('.mp4', '').split('_')[-1] for video_path in video_paths]
        for video_id, video_path in zip(video_ids, video_paths):
            video_path_full = f'{data_part_path}/{video_path}'
            all_video_paths[data_part][video_id] = video_path_full

    return all_video_paths

# Inference

In [ ]:
def create_directory(path):
    """Create a directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)

def predict_scenes_and_save(model, all_video_paths, save_dir):
    """Predict scenes for each video and save the results as JSON files."""
    create_directory(save_dir)

    for key, video_paths_dict in tqdm(all_video_paths.items()):      
        video_ids = sorted(video_paths_dict.keys())
        key_dir = os.path.join(save_dir, key)
        create_directory(key_dir)

        for video_id in tqdm(video_ids, desc=f"Processing {key}"):            
            video_path = video_paths_dict[video_id]
            _, single_frame_predictions, _ = model.predict_video(video_path)

            # Generate list of scenes from predictions, returns tuples of (start frame, end frame)
            scenes = model.predictions_to_scenes(single_frame_predictions)

            with open(os.path.join(key_dir, f"{video_id}.json"), 'w') as f:
                json.dump(scenes.tolist(), f)

In [ ]:
model = TransNetV2()
all_video_paths = parse_video_info(videos_dir)
predict_scenes_and_save(model, all_video_paths, save_dir)